In [1]:
from fastai.vision.all import *
import csv
import glob
from collections import namedtuple
from pathlib import Path

/home/josalhor/Desktop/eurecat-2022/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
csv_path = 'B/B/train.csv'
b2_path = Path('B/B/data')
path = b2_path

In [5]:
b2_attr = 'id path'
b2_pred = 'oil wide hole hor_fine2 hor_tense barring borrisol hor_fine pe'
BRow = namedtuple('BRow', b2_attr + ' ' + b2_pred)
b2_feat = b2_pred.split(' ')

In [14]:
rows = []
path_to_tp = {}
train_images = []

with open(csv_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        row[1] = Path('B/B/data') / Path(row[1])
        if not row[1].exists(): continue
        print(row)
        row = BRow(*row)
        pname = Path(row.path).name
        assert pname not in path_to_tp
        path_to_tp[pname] = row
        rows.append(row)
        train_images.append(Path(row.path))


['', Path('B/B/data/image_path'), 'Taca oli', 'Carrera', 'Agujero', 'Hilo fino horizontal2', 'Tensión horizontal', 'Barrado', 'Borrissol', 'Hilo fino horizontal', 'Punto escapado']
['8140', Path('B/B/data/4aaf06a9-frame0-00-08.95.jpg'), '0', '0', '0', '1', '0', '0', '0', '0', '0']
['11389', Path('B/B/data/19c8888a-frame0-00-15.68.jpg'), '0', '0', '0', '0', '0', '0', '0', '0', '0']
['1945', Path('B/B/data/aa6dfa7d-frame0-00-22.02.jpg'), '0', '0', '0', '0', '0', '0', '0', '0', '0']
['7981', Path('B/B/data/5e93a55a-frame0-00-12.03.jpg'), '0', '0', '0', '1', '0', '0', '0', '0', '0']
['9359', Path('B/B/data/d6f0df39-frame0-00-03.39.jpg'), '0', '0', '0', '0', '0', '0', '0', '1', '0']
['8391', Path('B/B/data/ba32fbbe-frame0-00-04.45.jpg'), '0', '0', '0', '1', '0', '0', '0', '0', '0']
['3299', Path('B/B/data/912a067f-frame0-00-38.28.jpg'), '0', '0', '0', '0', '0', '0', '0', '0', '0']
['1744', Path('B/B/data/8f1ffd6e-frame0-00-25.75.jpg'), '0', '0', '0', '0', '0', '0', '0', '0', '0']
['173', Pa

In [10]:
def oil_label(f):
    return path_to_tp[f].oil

def wide_label(f):
    return path_to_tp[f].wide

def hole_label(f):
    return path_to_tp[f].hole

def hor_fine2_label(f):
    return path_to_tp[f].hor_fine2

def barring_label(f):
    return path_to_tp[f].barring

def borrisol_label(f):
    return path_to_tp[f].borrisol


def hor_tense_label(f):
    return path_to_tp[f].hor_tense

def hor_fine_label(f):
    return path_to_tp[f].hor_fine

def pe_label(f):
    return path_to_tp[f].pe


In [8]:

def plot_metrics(filename, self: Recorder, nrows=None, ncols=None, figsize=None, **kwargs):
    metrics = np.stack(self.values)
    names = self.metric_names[1:-1]
    n = len(names) - 1
    if nrows is None and ncols is None:
        nrows = int(math.sqrt(n))
        ncols = int(np.ceil(n / nrows))
    elif nrows is None: nrows = int(np.ceil(n / ncols))
    elif ncols is None: ncols = int(np.ceil(n / nrows))
    figsize = figsize or (ncols * 6, nrows * 4)
    fig, axs = subplots(nrows, ncols, figsize=figsize, **kwargs)
    axs = [ax if i < n else ax.set_axis_off() for i, ax in enumerate(axs.flatten())][:n]
    for i, (name, ax) in enumerate(zip(names, [axs[0]] + axs)):
        ax.plot(metrics[:, i], color='#1f77b4' if i == 0 else '#ff7f0e', label='valid' if i > 0 else 'train')
        ax.set_title(name if i > 1 else 'losses')
        ax.legend(loc='best')
    plt.savefig(f'{filename}.png')

In [11]:

train_tp = [
    ('oil', oil_label),
    ('wide', wide_label),
    ('hole', hole_label),
    ('hor_fine2', hor_fine2_label),
    ('hor_tense', hor_tense_label),
    ('barring_fine', barring_label),
    ('borrisol_fine', borrisol_label),
    ('hor_fine', hor_fine_label),
    ('pe', pe_label),
]

In [12]:
def train(name, label_fn):
    print('Training', name)
    dls = ImageDataLoaders.from_name_func(path, train_images, label_fn, item_tfms=Resize(224))
    learn = vision_learner(dls, resnet18, metrics=[error_rate, accuracy, F1Score(), Recall(), Precision()])
    learn.fine_tune(10)
    plot_metrics(name, learn.recorder)
    learn.show_results()
    learn.export(name)
    return learn

In [16]:
for name, label_fn in train_tp:
    train(name, label_fn)

Training oil


FileNotFoundError: [Errno 2] No such file or directory: '/home/josalhor/Desktop/eurecat-2022/B/B/data/d7aab5e3-frame0-00-31.60.jpg'